In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from isolation_tree import IsolationTree

In [ ]:
n_components = 2
rng = jax.random.PRNGKey(42)
data = jax.random.normal(rng, (32, 2))
tree = IsolationTree.fit(rng, data, n_components)

X, Y = jnp.meshgrid(jnp.linspace(-5, 5, 1000), jnp.linspace(-5, 5, 1000))
coord = jnp.stack([X.flatten(), Y.flatten()]).T
data_paths = jax.vmap(tree.path)(data)
region_paths = jax.vmap(tree.path)(coord)

for depth, (data_path_ids, region_paths_ids) in enumerate(
    zip(data_paths[:, 1:].T, region_paths[:, 1:].T)
):
    print(f"data reached node {data_path_ids}")
    plt.figure(figsize=(6, 6))
    # scatter data color coded by region
    plt.scatter(
        data[:, 0], data[:, 1],s=200, c=data_path_ids,cmap="tab20", edgecolors="k",
        vmin=region_paths_ids.min(), vmax=region_paths_ids.min() + 19,
    )
    plt.scatter(data[:, 0], data[:, 1], marker="x", c="k", s=10)
    # imshow node regions
    plt.imshow(
        region_paths_ids.reshape(1000, 1000), cmap="tab20", vmin=region_paths_ids.min(),
        vmax=region_paths_ids.min() + 19, extent=(-5, 5, 5, -5),
    )
    plt.colorbar()
    plt.show()